Library Intallation

In [1]:
! pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached https://download.pytorch.org/whl/cu128/torch-2.8.0%2Bcu128-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu128/torchvision-0.23.0%2Bcu128-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
Using cached https://download.pytorch.org/whl/cu128/torch-2.8.0%2Bcu128-cp310-cp310-win_amd64.whl (3461.4 MB)
Using cached https://download.pytorch.org/whl/cu128/torchvision-0.23.0%2Bcu128-cp310-cp310-win_amd64.whl (7.5 MB)

   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   -----------------------

In [2]:
import torch
print(torch.__version__, torch.version.cuda)

2.8.0+cu128 12.8


In [3]:
# @title Install GNN dependencies
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

! pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
! pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu128.html
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     ---------------------------------------- 1.6/1.6 MB 22.1 MB/s  0:00:00


In [5]:
! pip install vmas

In [1]:
from vmas.simulator.scenario import BaseScenario

class MyScenario(BaseScenario):

    def make_world(self, batch_dim, device, **kwargs):
        raise NotImplementedError()

    def reset_world_at(self, env_index):
        raise NotImplementedError()

    def observation(self, agent):
        raise NotImplementedError()

    def reward(self, agent):
        raise NotImplementedError()

Make World

In [ ]:
import typing
from typing import Dict, List

import torch
from torch import Tensor

from vmas.simulator.core import Agent, Box, Landmark, Sphere, World
from vmas.simulator.dynamics.diff_drive import DiffDrive
from vmas.simulator.dynamics.holonomic import Holonomic
from vmas.simulator.dynamics.kinematic_bicycle import KinematicBicycle
from vmas.simulator.scenario import BaseScenario
from vmas.simulator.sensors import Lidar
from vmas.simulator.utils import (
    ANGULAR_FRICTION,
    Color,
    DRAG,
    LINEAR_FRICTION,
    ScenarioUtils,
)

if typing.TYPE_CHECKING:
    from vmas.simulator.rendering import Geom


def make_world(self, batch_dim: int, device: torch.device, **kwargs):
    ################
    # Scenario configuration
    ################
    self.plot_grid = False  # You can use this to plot a grid under the rendering for visualization purposes

    self.n_agents_holonomic = kwargs.pop(
        "n_agents_holonomic", 2
    )  # Number of agents with holonomic dynamics
    self.n_agents_diff_drive = kwargs.pop(
        "n_agents_diff_drive", 1
    )  # Number of agents with differential drive dynamics
    self.n_agents_car = kwargs.pop(
        "n_agents_car", 1
    )  # Number of agents with car dynamics
    self.n_agents = (
        self.n_agents_holonomic + self.n_agents_diff_drive + self.n_agents_car
    )
    self.n_obstacles = kwargs.pop("n_obstacles", 2)

    self.world_spawning_x = kwargs.pop(
        "world_spawning_x", 1
    )  # X-coordinate limit for entities spawning
    self.world_spawning_y = kwargs.pop(
        "world_spawning_y", 1
    )  # Y-coordinate limit for entities spawning

    self.comms_rendering_range = kwargs.pop(
        "comms_rendering_range", 0
    )  # Used for rendering communication lines between agents (just visual)
    self.lidar_range = kwargs.pop("lidar_range", 0.3)  # Range of the LIDAR sensor
    self.n_lidar_rays = kwargs.pop(
        "n_lidar_rays", 12
    )  # Number of LIDAR rays around the agent, each ray gives an observation between 0 and lidar_range

    self.shared_rew = kwargs.pop(
        "shared_rew", False
    )  # Whether the agents get a global or local reward for going to their goals
    self.final_reward = kwargs.pop(
        "final_reward", 0.01
    )  # Final reward that all the agents get when the scenario is done
    self.agent_collision_penalty = kwargs.pop(
        "agent_collision_penalty", -1
    )  # Penalty reward that an agent gets for colliding with another agent or obstacle

    self.agent_radius = kwargs.pop("agent_radius", 0.1)
    self.min_distance_between_entities = (
        self.agent_radius * 2 + 0.05
    )  # Minimum distance between entities at spawning time
    self.min_collision_distance = (
        0.005  # Minimum distance between entities for collision trigger
    )

    ScenarioUtils.check_kwargs_consumed(kwargs) # Warn if not all kwargs have been consumed


    ################
    # Make world
    ################
    world = World(
        batch_dim,  # Number of environments simulated
        device,  # Device for simulation
        substeps=5,  # Number of physical substeps (more yields more accurate but more expensive physics)
        collision_force=500,  # Paramneter to tune for collisions
        dt=0.1,  # Simulation timestep
        gravity=(0.0, 0.0),  # Customizable gravity
        drag=DRAG,  # Physics parameters
        linear_friction=LINEAR_FRICTION,  # Physics parameters
        angular_friction=ANGULAR_FRICTION,  # Physics parameters
        # There are many more....
    )

    ################
    # Add agents
    ################
    known_colors = [
        Color.BLUE,
        Color.ORANGE,
        Color.GREEN,
        Color.PINK,
        Color.PURPLE,
        Color.YELLOW,
        Color.RED,
    ]  # Colors for the first 7 agents
    colors = torch.randn(
        (max(self.n_agents - len(known_colors), 0), 3), device=device
    )  # Other colors if we have more agents are random

    self.goals = []  # We will store our agents' goal entities here for easy access
    for i in range(self.n_agents):
        color = (
            known_colors[i]
            if i < len(known_colors)
            else colors[i - len(known_colors)]
        )  # Get color for agent

        sensors = [
            Lidar(
                world,
                n_rays=self.n_lidar_rays,
                max_range=self.lidar_range,
                entity_filter=lambda e: isinstance(
                    e, Agent
                ),  # This makes sure that this lidar only percieves other agents
                angle_start=0.0,  # LIDAR angular ranges (we sense 360 degrees)
                angle_end=2
                * torch.pi,  # LIDAR angular ranges (we sense 360 degrees)
            )
        ]  # Agent LIDAR sensor

        if i < self.n_agents_holonomic:
            agent = Agent(
                name=f"holonomic_{i}",
                collide=True,
                color=color,
                render_action=True,
                sensors=sensors,
                shape=Sphere(radius=self.agent_radius),
                u_range=[1, 1],  # Ranges for actions
                u_multiplier=[1, 1],  # Action multipliers
                dynamics=Holonomic(),  # If you got to its class you can see it has 2 actions: force_x, and force_y
            )
        elif i < self.n_agents_holonomic + self.n_agents_diff_drive:
            agent = Agent(
                name=f"diff_drive_{i - self.n_agents_holonomic}",
                collide=True,
                color=color,
                render_action=True,
                sensors=sensors,
                shape=Sphere(radius=self.agent_radius),
                u_range=[1, 1],  # Ranges for actions
                u_multiplier=[0.5, 1],  # Action multipliers
                dynamics=DiffDrive(
                    world
                ),  # If you go to its class you can see it has 2 actions: forward velocity and angular velocity
            )
        else:
            max_steering_angle = torch.pi / 4
            width = self.agent_radius
            agent = Agent(
                name=f"car_{i-self.n_agents_holonomic-self.n_agents_diff_drive}",
                collide=True,
                color=color,
                render_action=True,
                sensors=sensors,
                shape=Box(length=self.agent_radius * 2, width=width),
                u_range=[1, max_steering_angle],
                u_multiplier=[0.5, 1],
                dynamics=KinematicBicycle(
                    world,
                    width=width,
                    l_f=self.agent_radius,  # Distance between the front axle and the center of gravity
                    l_r=self.agent_radius,  # Distance between the rear axle and the center of gravity
                    max_steering_angle=max_steering_angle,
                ),  # If you got to its class you can see it has 2 actions: forward velocity and steering angle
            )
        agent.pos_rew = torch.zeros(
            batch_dim, device=device
        )  # Tensor that will hold the position reward fo the agent
        agent.agent_collision_rew = (
            agent.pos_rew.clone()
        )  # Tensor that will hold the collision reward fo the agent

        world.add_agent(agent)  # Add the agent to the world

        ################
        # Add goals
        ################
        goal = Landmark(
            name=f"goal_{i}",
            collide=False,
            color=color,
        )
        world.add_landmark(goal)
        agent.goal = goal
        self.goals.append(goal)

    ################
    # Add obstacles
    ################
    self.obstacles = (
        []
    )  # We will store obstacles here for easy access
    for i in range(self.n_obstacles):
        obstacle = Landmark(
            name=f"obstacle_{i}",
            collide=True,
            color=Color.BLACK,
            shape=Sphere(radius=self.agent_radius * 2 / 3),
        )
        world.add_landmark(obstacle)
        self.obstacles.append(obstacle)

    self.pos_rew = torch.zeros(
        batch_dim, device=device
    )  # Tensor that will hold the global position reward
    self.final_rew = (
        self.pos_rew.clone()
    )  # Tensor that will hold the global done reward
    self.all_goal_reached = (
        self.pos_rew.clone()
    )  # Tensor indicating if all goals have been reached

    return world

MyScenario.make_world = make_world